# 0 Setting

In [ ]:
# Parameter Setting
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
folder_name = 'Research-(D5) Synthesized input model'
company_name = 'microsoft'
pretrained_model_name = 'model_microsoft.ckpt'

config = {
    'learning_rate': 2e-5,
    'batch_size': 32,
    'seq_length': 5,

    'shuffle': False,
    'criterion': torch.nn.CrossEntropyLoss(),
    'seed': 42,
    'valid_ratio': 0.2,
    'test_ratio': 0.2,
    'max_length': 512,
    'n_epochs': 3000,
    'early_stop': 50,
    'device': device,

    'h_text_size': 5,
    'h_c_size': 1,
    'h_news_size': 1,
    'h_tech_size': 6,
    'h_size': 32,
}

config_2 = {'input_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/data/2_'+company_name+'_for_model.csv',
            'save_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/model_saved/model.ckpt',
            # 'pretrained_model_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/premodel/' + pretrained_model_name,
            # 'continue_model_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/model_saved/model_1.ckpt'
            }

feature = [
    # X_1
    'input_ids',
    'attention_mask',
    'section_dummy',
    'publication_dummy',

    # X_2
    # 1. tech indicator
    # 'Open',
    # 'High',
    # 'Low',
    # 'Close',
    # 'Volume',
    # 'Dividends',
    # 'Stock Splits',
    'today_return',
    # 'today_return_cate',
    # 'Sma',
    # 'Rsi',
    # 'Kd',
    # 'Ema_12',
    # 'Ema_26',
    # 'Macd',
    # 'sentiment',

    # 2. market index
    # '^DJI',
    # '^GSPC',
    # '^NDX',
    # '^IXIC',
    # '^SOX',

    # y
    # '1_day_return',
    # '2_day_return',
    # '3_day_return',
    # '4_day_return',
    # '5_day_return',
    '1_day_return_cate',
    # '2_day_return_cate',
    # '3_day_return_cate',
    # '4_day_return_cate',
    # '5_day_return_cate',

    # Do not mark the datetime, it's for operation
    'datetime',
    ]

# All the news dataset
time_start = '2016-01-01T00:00:00'
time_end = '2020-04-02T00:00:00'

# time_start = '2016-01-01T00:00:00'
# time_end = '2016-04-31T00:00:00'

print(len(feature)-2)

5


## (1) Import

In [ ]:
# Google
from google.colab import drive
drive.mount('/content/drive')

# pip installation
!pip install transformers

# Basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# PyTorch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from transformers import BertTokenizer, BertModel, BertConfig

# others
from datetime import datetime, timedelta
from tqdm import tqdm
from torchsummary import summary
import ast

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.2 MB/s eta 0:00:00


In [ ]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set seed for reproducibility
same_seed(config['seed'])


In [ ]:
df = pd.read_csv(config_2['input_path'])
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'today_return', 'today_return_cate', '1_day_return', '2_day_return',
       '3_day_return', '4_day_return', '5_day_return', '1_day_return_cate',
       '2_day_return_cate', '3_day_return_cate', '4_day_return_cate',
       '5_day_return_cate', 'Sma', 'Rsi', 'Kd', 'Ema_12', 'Ema_26', 'Macd',
       'datetime', '^DJI', '^GSPC', '^NDX', '^IXIC', '^SOX', 'input_ids',
       'attention_mask', 'section_dummy', 'publication_dummy', 'sentiment'],
      dtype='object')

In [ ]:
df = df.sort_values(by='datetime', ascending=True)
df.shape

(1070, 36)

In [ ]:
# Only contain selected features
df = df[feature]
df.columns

Index(['input_ids', 'attention_mask', 'section_dummy', 'publication_dummy',
       'today_return', '1_day_return_cate', 'datetime'],
      dtype='object')

## (2) check nan

In [ ]:
df[df.isna().any(axis=1)]

,input_ids,attention_mask,section_dummy,publication_dummy,today_return,1_day_return_cate,datetime
1068,NaN,NaN,NaN,NaN,-0.005817,3,2020-04-01
1069,NaN,NaN,NaN,NaN,0.022389,3,2020-04-02


In [ ]:
df.isnull().sum()

input_ids            2
attention_mask       2
section_dummy        2
publication_dummy    2
today_return         0
1_day_return_cate    0
datetime             0
dtype: int64

In [ ]:
df = df.dropna()
df = df.reset_index(drop=True)
df.isnull().sum()

input_ids            0
attention_mask       0
section_dummy        0
publication_dummy    0
today_return         0
1_day_return_cate    0
datetime             0
dtype: int64

## (2) Time Period Selection

In [ ]:
# We use index to filter for time periods
df = df[(df['datetime']> time_start) & (df['datetime'] < time_end)]

# Drop datetime after using it
df.drop(columns=['datetime'], inplace=True)
df.shape

(1068, 6)

## (3) Transform str back to list

In [ ]:
# 将字符串转换回列表的函数
def string_to_list(s):
    return ast.literal_eval(s)

# 将列中的字符串转换回列表
df['input_ids'] = df['input_ids'].apply(string_to_list)
df['attention_mask'] = df['attention_mask'].apply(string_to_list)
df['section_dummy'] = df['section_dummy'].apply(string_to_list)
df['publication_dummy'] = df['publication_dummy'].apply(string_to_list)

In [ ]:
df['input_ids'][0][0]

[101,
 19102,
 1005,
 1055,
 9088,
 21628,
 21572,
 1055,
 2003,
 2066,
 2019,
 11924,
 13855,
 2770,
 3645,
 2184,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

## (3) List: Same amount of elements

In [ ]:
input_ids_list_length = len(df['input_ids'][0][0])
attention_mask_list_length = len(df['attention_mask'][0][0])
section_dummy_list_length = len(df['section_dummy'][0][0])
publication_dummy_list_length = len(df['publication_dummy'][0][0])

In [ ]:

# 找到最大的內部列表長度
max_inner_length = max(df['input_ids'].apply(len))

# 定義一個函數來填充內部列表，使其長度達到最大值
def pad_inner_list(lst, zero_list):
    while len(lst) < max_inner_length:
        lst.append(zero_list)  # 這裡可以填充任何你想要的值，例如 None

# 將 "input_ids" 列中的每個內部列表填充到最大長度
df['input_ids'].apply(pad_inner_list, zero_list=[0] * input_ids_list_length)
df['attention_mask'].apply(pad_inner_list, zero_list=[0] * attention_mask_list_length)
df['section_dummy'].apply(pad_inner_list, zero_list=[0] * section_dummy_list_length)
df['publication_dummy'].apply(pad_inner_list, zero_list=[0] * publication_dummy_list_length)
df

,input_ids,attention_mask,section_dummy,publication_dummy,today_return,1_day_return_cate
0,"[[101, 19102, 1005, 1055, 9088, 21628, 21572, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, ...",0.008837,2
1,"[[101, 16485, 2024, 9881, 2075, 2070, 5621, 68...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, ...",0.002185,0
2,"[[101, 2009, 1521, 1055, 2025, 3733, 2108, 105...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...","[[0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, ...",-0.004971,0
3,"[[101, 2003, 5082, 2633, 20727, 2046, 1996, 25...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, ...",-0.010057,2
4,"[[101, 2012, 2560, 2431, 1997, 2866, 5085, 203...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, ...",-0.000764,1
...,...,...,...,...,...,...
1063,"[[101, 2129, 2399, 5821, 8826, 11704, 27911, 3...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, ...",-0.013364,3
1064,"[[101, 7513, 3957, 4550, 9610, 2102, 2000, 561...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, ...",0.051954,0
1065,"[[101, 2822, 23707, 12058, 2015, 13463, 21887,...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, ...",-0.013509,3
1066,"[[101, 2235, 2449, 10940, 2071, 2468, 4539, 19...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, ...",0.051102,0


In [ ]:
df['input_ids']

0       [[101, 19102, 1005, 1055, 9088, 21628, 21572, ...
1       [[101, 16485, 2024, 9881, 2075, 2070, 5621, 68...
2       [[101, 2009, 1521, 1055, 2025, 3733, 2108, 105...
3       [[101, 2003, 5082, 2633, 20727, 2046, 1996, 25...
4       [[101, 2012, 2560, 2431, 1997, 2866, 5085, 203...
                              ...                        
1063    [[101, 2129, 2399, 5821, 8826, 11704, 27911, 3...
1064    [[101, 7513, 3957, 4550, 9610, 2102, 2000, 561...
1065    [[101, 2822, 23707, 12058, 2015, 13463, 21887,...
1066    [[101, 2235, 2449, 10940, 2071, 2468, 4539, 19...
1067    [[101, 2005, 1996, 4895, 5498, 10711, 3064, 19...
Name: input_ids, Length: 1068, dtype: object

In [ ]:
# 使用 apply 方法計算每個列表中元素的數量
temp = df['input_ids'].apply(len)

# 打印 DataFrame
temp

0       114
1       114
2       114
3       114
4       114
       ... 
1063    114
1064    114
1065    114
1066    114
1067    114
Name: input_ids, Length: 1068, dtype: int64

In [ ]:
# 計算"input_ids"列中所有list的平均長度
average_length = df['input_ids'].apply(len).mean()

# 計算"input_ids"列中最長的list的長度
max_length = df['input_ids'].apply(len).max()

# 計算"input_ids"列中最短的list的長度
min_length = df['input_ids'].apply(len).min()

# 打印結果
print(f"平均長度: {average_length}")
print(f"最長長度: {max_length}")
print(f"最短長度: {min_length}")


平均長度: 114.0
最長長度: 114
最短長度: 114


## int to float (section, publication)

In [ ]:
def recursive_convert_to_float(item):
    if isinstance(item, list):
        return [recursive_convert_to_float(x) if x is not None else None for x in item]
    else:
        return float(item) if item is not None else None

# 使用 apply 方法將函數應用於每個元素
df['section_dummy'] = df['section_dummy'].apply(recursive_convert_to_float)
df['publication_dummy'] = df['publication_dummy'].apply(recursive_convert_to_float)

In [ ]:
df['section_dummy'][0]

[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

## (4) Train_test_split

In [ ]:
# 1. Set up X, y
to_remove_list = ['datetime', '1_day_return_cate']

# Filter out values in to_remove_list
filtered_list = [x for x in feature if x not in to_remove_list]

X = df[filtered_list]
y = df['1_day_return_cate']

In [ ]:
# Check X, y shape
print('X:', X.shape)
print('y:', y.shape)

X: (1068, 5)
y: (1068,)


In [ ]:
# 2. train_test_split
# val dataset for final examination

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=config['test_ratio'], random_state=config['seed'], shuffle=config['shuffle'])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=config['valid_ratio'], random_state=config['seed'], shuffle=config['shuffle'])
X_train


,input_ids,attention_mask,section_dummy,publication_dummy,today_return
0,"[[101, 19102, 1005, 1055, 9088, 21628, 21572, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.008837
1,"[[101, 16485, 2024, 9881, 2075, 2070, 5621, 68...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.002185
2,"[[101, 2009, 1521, 1055, 2025, 3733, 2108, 105...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...","[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",-0.004971
3,"[[101, 2003, 5082, 2633, 20727, 2046, 1996, 25...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",-0.010057
4,"[[101, 2012, 2560, 2431, 1997, 2866, 5085, 203...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",-0.000764
...,...,...,...,...,...
678,"[[101, 3645, 2184, 2003, 5604, 1037, 3444, 200...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.002513
679,"[[101, 26269, 15686, 3006, 1011, 2877, 6627, 1...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...",0.007046
680,"[[101, 6950, 7523, 1037, 5418, 2000, 1996, 942...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.000088
681,"[[101, 1057, 1012, 1055, 1012, 6627, 7230, 323...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...",-0.013634


## (5) Scaler

In [ ]:
scale_feature = [
    # X_2
    # 1. tech indicator
    # 'Open',
    # 'High',
    # 'Low',
    # 'Close',
    # 'Volume',
    # 'Dividends',
    # 'Stock Splits',
    'today_return',
    # 'Today_trend_cate',
    # 'Sma',
    # 'Rsi',
    # 'Kd',
    # 'Ema_12',
    # 'Ema_26',
    # 'Macd',

    # 2. market index
    # '^DJI',
    # '^GSPC',
    # '^NDX',
    # '^IXIC',
    # '^SOX',
    # 'datetime'
    ]

def CustomScaler(X_train, X_val, X_test):
  scaler = MinMaxScaler()
  for i in scale_feature:

    # 對特定欄位進行標準化
    X_train_scaled = scaler.fit_transform(X_train[[i]])
    X_val_scaled = scaler.transform(X_val[[i]])
    X_test_scaled = scaler.transform(X_test[[i]])

    # 將標準化後的值重新賦值給 DataFrame
    X_train[i] = X_train_scaled
    X_val[i] = X_val_scaled
    X_test[i] = X_test_scaled

  return X_train, X_val, X_test

X_train, X_val, X_test = CustomScaler(X_train, X_val, X_test)

X_train

,input_ids,attention_mask,section_dummy,publication_dummy,today_return
0,"[[101, 19102, 1005, 1055, 9088, 21628, 21572, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.611251
1,"[[101, 16485, 2024, 9881, 2075, 2070, 5621, 68...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.550064
2,"[[101, 2009, 1521, 1055, 2025, 3733, 2108, 105...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...","[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.484248
3,"[[101, 2003, 5082, 2633, 20727, 2046, 1996, 25...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.437461
4,"[[101, 2012, 2560, 2431, 1997, 2866, 5085, 203...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.522944
...,...,...,...,...,...
678,"[[101, 3645, 2184, 2003, 5604, 1037, 3444, 200...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.553083
679,"[[101, 26269, 15686, 3006, 1011, 2877, 6627, 1...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...",0.594782
680,"[[101, 6950, 7523, 1037, 5418, 2000, 1996, 942...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.530781
681,"[[101, 1057, 1012, 1055, 1012, 6627, 7230, 323...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...",0.404562


## (6) Check number

In [ ]:
def calculate_class_stats(y):
    class_counts = y.value_counts()
    total_samples = len(y)
    class_ratios = class_counts / total_samples
    return class_counts, class_ratios

# 計算類別數量和比例
train_class_counts, train_class_ratios = calculate_class_stats(y_train)
val_class_counts, val_class_ratios = calculate_class_stats(y_val)
test_class_counts, test_class_ratios = calculate_class_stats(y_test)

# 創建包含數量和比例的 DataFrame
class_stats = pd.DataFrame({
    'Train Count': train_class_counts,
    'Train Ratio': train_class_ratios,
    'Validation Count': val_class_counts,
    'Validation Ratio': val_class_ratios,
    'Test Count': test_class_counts,
    'Test Ratio': test_class_ratios
})

# 打印 DataFrame
print(class_stats)


   Train Count  Train Ratio  Validation Count  Validation Ratio  Test Count  \
0           88     0.128843                36          0.210526          43   
1          223     0.326501                40          0.233918          51   
2          238     0.348463                54          0.315789          60   
3          134     0.196193                41          0.239766          60   

   Test Ratio  
0    0.200935  
1    0.238318  
2    0.280374  
3    0.280374  


In [ ]:
# Time period
print('Time Period')
print('From:', time_start)
print('To:', time_end, '\n')

# Sample size
print('Sample size:', X.shape[0])
print('Feature:', X.columns, '\n')
print('Target:', y.name, '\n')
print('Train: Val: Test =', X_train.shape[0], X_test.shape[0], X_val.shape[0])

Time Period
From: 2016-01-01T00:00:00
To: 2020-04-02T00:00:00 

Sample size: 1068
Feature: Index(['input_ids', 'attention_mask', 'section_dummy', 'publication_dummy',
       'today_return'],
      dtype='object') 

Target: 1_day_return_cate 

Train: Val: Test = 683 214 171


# Model

## (1) Dataset & Dataloader

In [ ]:
# Dataset
X_1 =['input_ids', 'attention_mask', 'section_dummy', 'publication_dummy']


class CustomDataset(Dataset):
    def __init__(self, X, y, config):
        # X_1
        self.input_ids = X['input_ids']
        self.attention_mask = X['attention_mask']
        self.section = X['section_dummy']
        self.publication = X['publication_dummy']

        # X_2
        self.X_2 = torch.tensor(X.drop(columns=X_1).values, dtype=torch.float)

        # y
        self.y = torch.tensor(y.values, dtype=torch.long)

        # other setting
        self.len = X.shape[0]
        self.seq_length = config['seq_length']

    def __getitem__(self,idx):
        # X_1
        input_ids_list = self.input_ids[idx : idx + self.seq_length].tolist() # All to list
        input_ids = torch.tensor(input_ids_list) # Then to tensor
        attention_mask_list = self.attention_mask[idx : idx + self.seq_length].tolist()
        attention_mask = torch.tensor(attention_mask_list)
        section_list = self.section[idx : idx + self.seq_length].tolist()
        section = torch.tensor(section_list)
        publication_list = self.publication[idx : idx + self.seq_length].tolist()
        publication = torch.tensor(publication_list)

        # X_2
        X_2 = self.X_2[idx : idx + self.seq_length]

        # 3. y
        y = self.y[idx + self.seq_length - 1]

        return input_ids, attention_mask, section, publication, X_2, y

    def __len__(self):
        return self.len - self.seq_length

In [ ]:
# DataLoader
train_dataset = CustomDataset(X_train, y_train, config)
val_dataset = CustomDataset(X_val, y_val, config)
test_dataset = CustomDataset(X_test, y_test, config)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], drop_last=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], drop_last=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], drop_last=True, pin_memory=True)

# Check loader output
for batch in train_loader:
    input_ids, attention_mask, section, publication, X_2, y = batch

    # 打印批次数据的形状，以确保它们符合预期
    print("Input IDs shape:", input_ids.shape)
    print("Attention Mask shape:", attention_mask.shape)
    print("Section shape:", section.shape)
    print("Publication shape:", publication.shape)
    print("X_2 shape:", X_2.shape)
    print("Labels shape:", y.shape)

    # print("Input IDs:", input_ids)
    # print("Attention Mask:", attention_mask)
    # print("Section:", section)
    # print("Publication:", publication)
    # print("X_2:", X_2)
    # print("Labels:", y)

    break


Input IDs shape: torch.Size([32, 5, 114, 32])
Attention Mask shape: torch.Size([32, 5, 114, 32])
Section shape: torch.Size([32, 5, 114, 9])
Publication shape: torch.Size([32, 5, 114, 12])
X_2 shape: torch.Size([32, 5, 1])
Labels shape: torch.Size([32])


## (2) Model Architecture

### 1 Premodel

In [ ]:
# # model_microsoft
# class PreModel(nn.Module):
#     def __init__(self, base_model):
#         super(PreModel, self).__init__()
#         self.base_model = BertModel.from_pretrained('bert-base-uncased')

#         # 2. FC layers
#         self.fc1 = nn.Linear(base_model.config.hidden_size, 256)
#         self.fc2 = nn.Linear(256, 64)
#         self.fc3 = nn.Linear(64, 1)
#         # self.fc4 = nn.Linear(256, 64)
#         # self.fc5 = nn.Linear(64, 5)

#     def forward(self, input_ids, attention_mask):
#         out = self.base_model(input_ids=input_ids, attention_mask=attention_mask)

#         out = out.pooler_output
#         out = self.fc1(out)
#         out = self.fc2(out)
#         out = self.fc3(out)
#         # out = self.fc4(out)
#         # out = self.fc5(out)

#         return out

### 2 Prediction model

In [ ]:
# # New structure: Save Computation
# class MyModel(nn.Module):
#     def __init__(self, base_model, config, element_size, section_length, publication_length, X_2_length, batch_size):
#         super(MyModel, self).__init__()
#         self.seq_length = config['seq_length']
#         self.batch_size = batch_size
#         self.element_size = element_size
#         self.abandon_tensor = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#                               0, 0, 0, 0, 0, 0, 0, 0], device=device)

#         self.section_length = section_length
#         self.publication_length = publication_length
#         self.config = config

#         # 1. News
#         # text
#         self.base_model = base_model
#         self.fc1 = nn.Linear(768, 64)
#         self.fc2 = nn.Linear(64, config['h_text_size'])

#         # c
#         self.fc_h_c = nn.Linear(section_length + publication_length, config['h_c_size'])

#         # news (concated)
#         self.fc_h_news = nn.Linear(config['h_text_size'] + config['h_c_size'], config['h_news_size'])

#         # 2. Indicator
#         # tech
#         self.fc_h_tech = nn.Linear(X_2_length, config['h_tech_size'])

#         # 1&2. converge
#         # news + tech
#         self.fc_h = nn.Linear(config['h_news_size'] + config['h_tech_size'], config['h_size'])

#         # 3. LSTM
#         self.lstm_1 = nn.LSTM(config['h_size'], 32, dropout=0.2, num_layers=4, batch_first=True, bidirectional=False)
#         # self.lstm_2 = nn.LSTM(32, 64, num_layers=1, batch_first=True, bidirectional=False)
#         # self.lstm_3 = nn.LSTM(64, 16, num_layers=1, batch_first=True, bidirectional=False)
#         # self.dropout = nn.Dropout(0.2)
#         self.sequential = nn.Sequential(
#             nn.Linear(32, 3)
#             # nn.Linear(16, 1)
#         )
#         self.dropout = nn.Dropout(0.2)


#     def forward(self, input_ids, attention_mask, section, publication, X_2):
#         # 1. News
#         flattened_input_ids = input_ids.view(-1, 32)
#         flattened_attention_mask = attention_mask.view(-1, 32)
#         flattened_section = section.view(-1, self.section_length)
#         flattened_publication = publication.view(-1, self.publication_length)

#         e_list = []
#         for i in range(0, flattened_input_ids.size(0), self.element_size):
#           # 获取当前组的子张量
#           sub_input_ids = flattened_input_ids[i:i+self.element_size]
#           sub_attention_mask = flattened_attention_mask[i:i+self.element_size]
#           sub_section = flattened_section[i:i+self.element_size]
#           sub_publication = flattened_publication[i:i+self.element_size]

#           non_zero_mask = (sub_input_ids != 0).any(dim=1)
#           non_zero_input_ids = sub_input_ids[non_zero_mask]
#           non_zero_attention_mask = sub_attention_mask[non_zero_mask]
#           non_zero_section = sub_section[non_zero_mask]
#           non_zero_publication = sub_publication[non_zero_mask]

#           # input_ids, attention_mask
#           out = self.base_model(input_ids=non_zero_input_ids, attention_mask=non_zero_attention_mask)
#           out = out.pooler_output
#           out = self.fc1(out)
#           h_text = self.fc2(out)

#           # section, publication
#           out = torch.cat([non_zero_section, non_zero_publication], dim=1)
#           h_c = self.fc_h_c(out)

#           # h_news
#           out = torch.cat([h_text, h_c], dim=1)
#           out = self.fc_h_news(out)
#           h_news = self.dropout(out)
#           element_mean = torch.mean(h_news, dim=0)
#           e_list.append(element_mean)

#         temp_tensor = torch.stack(e_list)
#         b_tensor = temp_tensor.view(self.batch_size, self.seq_length, self.config['h_news_size'])

#         # 2. Indicator
#         # h_tech
#         # h_tech = self.fc_h_tech(X_2)
#         h_tech = X_2

#         # h
#         out = torch.cat([b_tensor, h_tech], dim=2)
#         h = self.fc_h(out)

#         # 3. LSTM
#         out, _ = self.lstm_1(h)
#         out = out[:, -1, :]  # Get the last one of LSTM output for prediction of next-term

#         final_out = self.sequential(out)

#         return final_out


3. Simplified model
without c

In [ ]:
# New structure: Save Computation
class MyModel(nn.Module):
    def __init__(self, base_model, config, element_size, section_length, publication_length, X_2_length, batch_size):
        super(MyModel, self).__init__()
        self.seq_length = config['seq_length']
        self.batch_size = batch_size
        self.element_size = element_size
        self.abandon_tensor = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                              0, 0, 0, 0, 0, 0, 0, 0], device=device)

        self.section_length = section_length
        self.publication_length = publication_length
        self.config = config

        # 1. News
        # text
        self.base_model = base_model
        self.fc1 = nn.Linear(768, config['h_text_size'])
        # self.fc2 = nn.Linear(256, )


        # 1&2. converge
        # news + tech
        # self.fc_h = nn.Linear(config['h_text_size'] + config['h_tech_size'], config['h_size'])

        # 3. LSTM
        self.lstm_1 = nn.LSTM(config['h_text_size']+X_2_length, 32, dropout=0.2, num_layers=2, batch_first=True, bidirectional=False)
        self.sequential = nn.Sequential(
            nn.Linear(32, 4)
        )
        self.dropout = nn.Dropout(0.2)


    def forward(self, input_ids, attention_mask, section, publication, X_2):
        # 1. News
        flattened_input_ids = input_ids.view(-1, 32)
        flattened_attention_mask = attention_mask.view(-1, 32)
        flattened_section = section.view(-1, self.section_length)
        flattened_publication = publication.view(-1, self.publication_length)

        e_list = []
        for i in range(0, flattened_input_ids.size(0), self.element_size):
          # 获取当前组的子张量
          sub_input_ids = flattened_input_ids[i:i+self.element_size]
          sub_attention_mask = flattened_attention_mask[i:i+self.element_size]
          sub_section = flattened_section[i:i+self.element_size]
          sub_publication = flattened_publication[i:i+self.element_size]

          non_zero_mask = (sub_input_ids != 0).any(dim=1)
          non_zero_input_ids = sub_input_ids[non_zero_mask]
          non_zero_attention_mask = sub_attention_mask[non_zero_mask]
          non_zero_section = sub_section[non_zero_mask]
          non_zero_publication = sub_publication[non_zero_mask]

          # input_ids, attention_mask
          out = self.base_model(input_ids=non_zero_input_ids, attention_mask=non_zero_attention_mask)
          out = out.pooler_output
          # out = self.dropout(out)
          h_text = self.fc1(out)
          # out = self.dropout(out)
          #  = self.fc2(out)

          element_mean = torch.mean(h_text, dim=0)
          e_list.append(element_mean)

        temp_tensor = torch.stack(e_list)
        b_tensor = temp_tensor.view(self.batch_size, self.seq_length, self.config['h_text_size'])

        # 2. Indicator
        # h_tech
        h_tech = X_2

        # h
        h = torch.cat([b_tensor, h_tech], dim=2)

        # 3. LSTM
        out, _ = self.lstm_1(h)
        out = out[:, -1, :]  # Get the last one of LSTM output for prediction of next-term

        final_out = self.sequential(out)

        return final_out


## (4) Load Model

### 1. Load pretrain model

In [ ]:
# 載入預訓練模型

# base_model = PreModel(base_model)
# base_model.load_state_dict(torch.load(config_2['pretrained_model_path']))
bert_config = BertConfig(hidden_dropout_prob=0.2)
base_model = BertModel.from_pretrained('bert-base-uncased', config=bert_config)


# Parameter
element_size = len(df['input_ids'][0])  # 114
section_length = len(df['section_dummy'][0][0])
publication_length = len(df['publication_dummy'][0][0])
X_2_length = len(feature) - 6


### 2. Initiate Model

In [ ]:
model = MyModel(base_model, config, element_size, section_length, publication_length, X_2_length, config['batch_size'])

model.to(device)

MyModel(
  (base_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

### Extra: Contunue training

In [ ]:
# model = MyModel(base_model, config, section_length, publication_length, X_2_length)
# model.load_state_dict(torch.load(config_2['continue_model_path']))
# model.to(device)

# # 分段訓練
# trainer2(model, train_loader, val_loader, config, device)
# trainer1(model, train_loader, val_loader, config, device)

## (5) Require_grad

In [ ]:
# Freeze all layers
for param in model.base_model.parameters():
  param.requires_grad = False

# Unfreeze part of layers
# for param in model.base_model.encoder.layer[6].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[7].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[8].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[9].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[10].parameters():
#     param.requires_grad = True

for param in model.base_model.encoder.layer[11].parameters():
    param.requires_grad = True

# for param in model.base_model.fc1.parameters():
#     param.requires_grad = True

# for param in model.base_model.fc2.parameters():
#     param.requires_grad = True

# for param in model.base_model.fc3.parameters():
    # param.requires_grad = True

# Check requires_grad status
for name, param in model.named_parameters():
    print(name, param.requires_grad)

base_model.embeddings.word_embeddings.weight False
base_model.embeddings.position_embeddings.weight False
base_model.embeddings.token_type_embeddings.weight False
base_model.embeddings.LayerNorm.weight False
base_model.embeddings.LayerNorm.bias False
base_model.encoder.layer.0.attention.self.query.weight False
base_model.encoder.layer.0.attention.self.query.bias False
base_model.encoder.layer.0.attention.self.key.weight False
base_model.encoder.layer.0.attention.self.key.bias False
base_model.encoder.layer.0.attention.self.value.weight False
base_model.encoder.layer.0.attention.self.value.bias False
base_model.encoder.layer.0.attention.output.dense.weight False
base_model.encoder.layer.0.attention.output.dense.bias False
base_model.encoder.layer.0.attention.output.LayerNorm.weight False
base_model.encoder.layer.0.attention.output.LayerNorm.bias False
base_model.encoder.layer.0.intermediate.dense.weight False
base_model.encoder.layer.0.intermediate.dense.bias False
base_model.encoder.la

# Training

In [ ]:
def trainer(model, train_loader, val_loader, config, device):

    criterion = config['criterion']
    # optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'])


    writer = SummaryWriter()  # Writer of tensoboard.
    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    # 1. Training
    for epoch in range(n_epochs):
      model.train()  # Set the model to training mode
      loss_record = []

      train_pbar = tqdm(train_loader, position=0, leave=True)  # tqdm is a package to visualize your training progress.
      for input_ids, attention_mask, section, publication, X_2, y in train_loader:
        optimizer.zero_grad()  # Set gradient to zero

        # Forward pass
        input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
        pred = model(input_ids, attention_mask, section, publication, X_2)
        loss = criterion(pred, y)
        loss.backward()                     # Compute gradient(backpropagation).
        optimizer.step()                    # Update parameters.
        step += 1
        loss_record.append(loss.detach().item())

        # Display current epoch number and loss on tqdm progress bar.
        train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
        train_pbar.set_postfix({'loss': loss.detach().item()})

      mean_train_loss = sum(loss_record)/len(loss_record)
      writer.add_scalar('Loss/train', mean_train_loss, step)

      # 2. Evaluation
      model.eval() # Set your model to evaluation mode.
      loss_record = []
      predicted_labels_list = []
      targets_list = []
      for input_ids, attention_mask, section, publication, X_2, y in val_loader:
          input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
          with torch.no_grad():
              pred = model(input_ids, attention_mask, section, publication, X_2)
              _, predicted = torch.max(pred, 1)
              loss = criterion(pred, y)
              predicted_labels_list.extend(predicted.tolist())
              targets_list.extend(y.tolist())
              loss_record.append(loss.item())
      accuracy = accuracy_score(targets_list, predicted_labels_list)

      # Mean
      mean_valid_loss = sum(loss_record)/len(loss_record)
      print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}, Val Acc: {accuracy:.4f}')
      writer.add_scalar('Loss/valid', mean_valid_loss, step)

      # 3. Judge of saving model
      if mean_valid_loss < best_loss:
          best_loss = mean_valid_loss
          torch.save(model.state_dict(), config_2['save_path']) # Save your best model
          print('Saving model with loss {:.3f}...'.format(best_loss))
          early_stop_count = 0
      else:
          early_stop_count += 1

      if early_stop_count >= config['early_stop']:
          print('\nModel is not improving, so we halt the training session.')
          return


In [ ]:
# 全部訓練
trainer(model, train_loader, val_loader, config, device)


Epoch [1/3000]:   0%|          | 0/21 [02:09<?, ?it/s, loss=1.38]

Epoch [1/3000]: Train loss: 1.3892, Valid loss: 1.3922, Val Acc: 0.2313
Saving model with loss 1.392...


Epoch [2/3000]:   0%|          | 0/21 [01:02<?, ?it/s, loss=1.36]

KeyboardInterrupt: ignored

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

# Evaluate

In [ ]:
# # Evaluation Dataloader
# con_train_loader = DataLoader(train_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)
# con_val_loader = DataLoader(val_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)
# con_test_loader = DataLoader(test_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)

# # Evaluation mode
# model.eval()
# accuracy_list = []

# # 1. Train part
# with torch.no_grad():
#     predicted_labels_list = []
#     targets_list = []
#     for input_ids, attention_mask, section, publication, X_2, y in con_train_loader:
#         input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#         outputs = model(input_ids, attention_mask, section, publication, X_2)
#         _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

#         predicted_labels_list.extend(predicted_labels.tolist())
#         targets_list.extend(y.tolist())

#     # 计算准确率
#     accuracy = accuracy_score(targets_list, predicted_labels_list)
#     accuracy_list.append(accuracy)

# print('=====================================================================================================================')
# print('Training Result:')
# print(classification_report(targets_list, predicted_labels_list))
# print(confusion_matrix(targets_list, predicted_labels_list), '\n')

# # 2. Val part
# with torch.no_grad():
#     predicted_labels_list = []
#     targets_list = []
#     for input_ids, attention_mask, section, publication, X_2, y in con_val_loader:
#         input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#         outputs = model(input_ids, attention_mask, section, publication, X_2)
#         _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

#         predicted_labels_list.extend(predicted_labels.tolist())
#         targets_list.extend(y.tolist())

#     # 计算准确率
#     accuracy = accuracy_score(targets_list, predicted_labels_list)
#     accuracy_list.append(accuracy)

# print('=====================================================')
# print('Val Result:')
# print(classification_report(targets_list, predicted_labels_list))
# print(confusion_matrix(targets_list, predicted_labels_list))


# # 3. Test part
# with torch.no_grad():
#     predicted_labels_list = []
#     targets_list = []
#     for input_ids, attention_mask, section, publication, X_2, y in con_test_loader:
#         input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#         outputs = model(input_ids, attention_mask, section, publication, X_2)
#         _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

#         predicted_labels_list.extend(predicted_labels.tolist())
#         targets_list.extend(y.tolist())

#     # 计算准确率
#     accuracy = accuracy_score(targets_list, predicted_labels_list)
#     accuracy_list.append(accuracy)

# print('=====================================================')
# print('Testing Result:')
# print(classification_report(targets_list, predicted_labels_list))
# print(confusion_matrix(targets_list, predicted_labels_list))

# print('=====================================================', '\n')
# print("Accuracy [Train, Val, Test]: ", accuracy_list, '\n')
# print(model, '\n')
# print('Config: ', config, '\n')
# print('Feature: ', feature)
# print('time_start: ', time_start, 'time_end: ', time_end)

In [ ]:
# Evaluation Dataloader
con_train_loader = DataLoader(train_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)
con_val_loader = DataLoader(val_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)
con_test_loader = DataLoader(test_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)

model = MyModel(base_model, config, element_size, section_length, publication_length, X_2_length, batch_size=1)
model.load_state_dict(torch.load(config_2['save_path']))
model.to(device)

# Evaluation mode
model.eval()

accuracy_list = []

# 1. Train part
with torch.no_grad():
    predicted_labels_list = []
    targets_list = []
    for input_ids, attention_mask, section, publication, X_2, y in con_train_loader:
        input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
        outputs = model(input_ids, attention_mask, section, publication, X_2)
        _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

    # 计算准确率
    accuracy = accuracy_score(targets_list, predicted_labels_list)
    accuracy_list.append(accuracy)

print('=====================================================================================================================')
print('Training Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list), '\n')

# 2. Val part
with torch.no_grad():
    predicted_labels_list = []
    targets_list = []
    for input_ids, attention_mask, section, publication, X_2, y in con_val_loader:
        input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
        outputs = model(input_ids, attention_mask, section, publication, X_2)
        _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

    # 计算准确率
    accuracy = accuracy_score(targets_list, predicted_labels_list)
    accuracy_list.append(accuracy)

print('=====================================================')
print('Val Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list))


# 3. Test part
with torch.no_grad():
    predicted_labels_list = []
    targets_list = []
    for input_ids, attention_mask, section, publication, X_2, y in con_test_loader:
        input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
        outputs = model(input_ids, attention_mask, section, publication, X_2)
        _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

    # 计算准确率
    accuracy = accuracy_score(targets_list, predicted_labels_list)
    accuracy_list.append(accuracy)

print('=====================================================')
print('Testing Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list))

print('=====================================================', '\n')
print("Accuracy [Train, Val, Test]: ", accuracy_list, '\n')
# print(model, '\n')
print('Config: ', config, '\n')
print('Feature: ', feature)
print('time_start: ', time_start, 'time_end: ', time_end)

In [ ]:
# # Evaluation Dataloader
# con_train_loader = DataLoader(train_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)
# con_val_loader = DataLoader(val_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)
# con_test_loader = DataLoader(test_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)

# # Evaluation mode
# model.eval()
# accuracy_list = []
# threshold = 0.5

# # 1. Train part
# predicted_labels_list = []
# targets_list = []

# with torch.no_grad():
#     for input_ids, attention_mask, section, publication, X_2, y in con_train_loader:
#         input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#         outputs = model(input_ids, attention_mask, section, publication, X_2)
#         # _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引
#         predicted_labels = (outputs >= threshold).float()

#         predicted_labels_list.extend(predicted_labels.tolist())
#         targets_list.extend(y.tolist())

# accuracy = accuracy_score(targets_list, predicted_labels_list)

# print('=====================================================================================================================')
# print('Training Result:')
# print(classification_report(targets_list, predicted_labels_list))
# print(confusion_matrix(targets_list, predicted_labels_list), '\n')

# # 2. Val part
# predicted_labels_list = []
# targets_list = []

# with torch.no_grad():
#     for input_ids, attention_mask, section, publication, X_2, y in con_val_loader:
#         input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#         outputs = model(input_ids, attention_mask, section, publication, X_2)
#         # _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引
#         predicted_labels = (outputs >= 0.5).float()

#         predicted_labels_list.extend(predicted_labels.tolist())
#         targets_list.extend(y.tolist())

# accuracy = accuracy_score(targets_list, predicted_labels_list)

# print('=====================================================')
# print('Val Result:')
# print(classification_report(targets_list, predicted_labels_list))
# print(confusion_matrix(targets_list, predicted_labels_list))


# # 3. Test part
# predicted_labels_list = []
# targets_list = []

# with torch.no_grad():
#     for input_ids, attention_mask, section, publication, X_2, y in con_test_loader:
#         input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#         outputs = model(input_ids, attention_mask, section, publication, X_2)
#         # _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引
#         predicted_labels = (outputs >= 0.5).float()

#         predicted_labels_list.extend(predicted_labels.tolist())
#         targets_list.extend(y.tolist())

# accuracy = accuracy_score(targets_list, predicted_labels_list)

# print('=====================================================')
# print('Testing Result:')
# print(classification_report(targets_list, predicted_labels_list))
# print(confusion_matrix(targets_list, predicted_labels_list))

# print('=====================================================', '\n')
# print("Accuracy [Train, Val, Test]: ", accuracy_list, '\n')
# # print(model, '\n')
# print('Config: ', config, '\n')
# print('Feature: ', feature)
# print('time_start: ', time_start, 'time_end: ', time_end)

In [ ]:
# # Evaluation Dataloader
# con_train_loader = DataLoader(train_dataset, batch_size=1, shuffle=config['shuffle'], pin_memory=True)
# con_val_loader = DataLoader(val_dataset, batch_size=1, shuffle=config['shuffle'], pin_memory=True)
# con_test_loader = DataLoader(test_dataset, batch_size=1, shuffle=config['shuffle'], pin_memory=True)

# model = MyModel(base_model, config, element_size, section_length, publication_length, X_2_length, batch_size=1)
# model.load_state_dict(torch.load(config_2['save_path']))
# model.to(device)

# # Evaluation mode
# model.eval()
# accuracy_list = []
# threshold = 0.5

# # 1. Train part
# predicted_labels_list = []
# targets_list = []

# with torch.no_grad():
#     for input_ids, attention_mask, section, publication, X_2, y in con_train_loader:
#         input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#         outputs = model(input_ids, attention_mask, section, publication, X_2)
#         # _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引
#         predicted_labels = (outputs >= threshold).float()

#         predicted_labels_list.extend(predicted_labels.tolist())
#         targets_list.extend(y.tolist())

# accuracy = accuracy_score(targets_list, predicted_labels_list)

# print('=====================================================================================================================')
# print('Training Result:')
# print(classification_report(targets_list, predicted_labels_list))
# print(confusion_matrix(targets_list, predicted_labels_list), '\n')

# # 2. Val part
# predicted_labels_list = []
# targets_list = []

# with torch.no_grad():
#     for input_ids, attention_mask, section, publication, X_2, y in con_val_loader:
#         input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#         outputs = model(input_ids, attention_mask, section, publication, X_2)
#         # _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引
#         predicted_labels = (outputs >= 0.5).float()

#         predicted_labels_list.extend(predicted_labels.tolist())
#         targets_list.extend(y.tolist())

# accuracy = accuracy_score(targets_list, predicted_labels_list)

# print('=====================================================')
# print('Val Result:')
# print(classification_report(targets_list, predicted_labels_list))
# print(confusion_matrix(targets_list, predicted_labels_list))


# # 3. Test part
# predicted_labels_list = []
# targets_list = []

# with torch.no_grad():
#     for input_ids, attention_mask, section, publication, X_2, y in con_test_loader:
#         input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#         outputs = model(input_ids, attention_mask, section, publication, X_2)
#         # _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引
#         predicted_labels = (outputs >= 0.5).float()

#         predicted_labels_list.extend(predicted_labels.tolist())
#         targets_list.extend(y.tolist())

# accuracy = accuracy_score(targets_list, predicted_labels_list)

# print('=====================================================')
# print('Testing Result:')
# print(classification_report(targets_list, predicted_labels_list))
# print(confusion_matrix(targets_list, predicted_labels_list))

# print('=====================================================', '\n')
# print("Accuracy [Train, Val, Test]: ", accuracy_list, '\n')
# print('Config: ', config, '\n')
# print('Feature: ', feature)
# print('time_start: ', time_start, 'time_end: ', time_end)

## (2) Thresholds

In [ ]:
# # function of evaluation
# def evaluate_model(model, dataloader, threshold=0.5, device='cuda'):
#     predicted_labels_list = []
#     targets_list = []

#     with torch.no_grad():
#         for input_ids, attention_mask, section, publication, X_2, y in dataloader:
#             input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#             outputs = model(input_ids, attention_mask, section, publication, X_2)
#             predicted_labels = (outputs >= threshold).float()

#             predicted_labels_list.extend(predicted_labels.tolist())
#             targets_list.extend(y.tolist())

#     accuracy = accuracy_score(targets_list, predicted_labels_list)

#     print('=====================================================================================================================')
#     print(str(dataloader), ' Result:')
#     print(classification_report(targets_list, predicted_labels_list))
#     print(confusion_matrix(targets_list, predicted_labels_list), '\n')

In [ ]:
# accuracy_list = []
# threshold = 0.51

# evaluate_model(model, con_train_loader, threshold=threshold)
# evaluate_model(model, con_val_loader, threshold=threshold)
# evaluate_model(model, con_test_loader, threshold=threshold)

In [ ]:
# accuracy_list = []
# threshold = 0.52

# evaluate_model(model, con_train_loader, threshold=threshold)
# evaluate_model(model, con_val_loader, threshold=threshold)
# evaluate_model(model, con_test_loader, threshold=threshold)

In [ ]:
# accuracy_list = []
# threshold = 0.53

# evaluate_model(model, con_train_loader, threshold=threshold)
# evaluate_model(model, con_val_loader, threshold=threshold)
# evaluate_model(model, con_test_loader, threshold=threshold)

In [ ]:
# accuracy_list = []
# threshold = 0.54

# evaluate_model(model, con_train_loader, threshold=threshold)
# evaluate_model(model, con_val_loader, threshold=threshold)
# evaluate_model(model, con_test_loader, threshold=threshold)

In [ ]:
# accuracy_list = []
# threshold = 0.55

# evaluate_model(model, con_train_loader, threshold=threshold)
# evaluate_model(model, con_val_loader, threshold=threshold)
# evaluate_model(model, con_test_loader, threshold=threshold)

In [ ]:
# accuracy_list = []
# threshold = 0.56

# evaluate_model(model, con_train_loader, threshold=threshold)
# evaluate_model(model, con_val_loader, threshold=threshold)
# evaluate_model(model, con_test_loader, threshold=threshold)

In [ ]:
# accuracy_list = []
# threshold = 0.57

# evaluate_model(model, con_train_loader, threshold=threshold)
# evaluate_model(model, con_val_loader, threshold=threshold)
# evaluate_model(model, con_test_loader, threshold=threshold)

In [ ]:
# accuracy_list = []
# threshold = 0.58

# evaluate_model(model, con_train_loader, threshold=threshold)
# evaluate_model(model, con_val_loader, threshold=threshold)
# evaluate_model(model, con_test_loader, threshold=threshold)

# Benchmark

In [ ]:
# class BenchmarkModel(nn.Module):
#     def __init__(self, config, section_length, publication_length, X_2_length):
#         super(BenchmarkModel, self).__init__()
#         # tech
#         self.fc_h_tech = nn.Linear(X_2_length, config['h_tech_size'])

#         # 3. LSTM
#         self.lstm_1 = nn.LSTM(config['h_size'], 2, num_layers=1, batch_first=True, bidirectional=False)
#         self.sequential = nn.Sequential(
#             nn.Linear(2, 1)
#         )
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, input_ids, attention_mask, section, publication, X_2):
#         # h_tech
#         h_tech = self.fc_h_tech(X_2)

#         # 3. LSTM
#         out, _ = self.lstm_1(h_tech)
#         out = out[:, -1, :]  # Get the last one of LSTM output for prediction of next-term
#         # print('out 1(last layer of LSTM): ', out.shape, out)
#         out = self.sequential(out)
#         # print('out 2(after sequnetial): ',out.shape , out)
#         final_out = self.sigmoid(out)
#         # print('final_out: ', final_out.shape, final_out)

#         return final_out
